In [1]:
import samna
import time

print(samna.__version__)
    
d= samna.device.get_unopened_devices()
dk = samna.device.open_device(d[0])

pm = dk.get_power_monitor()
power_module = dk.get_power_module()

buf = samna.graph.sink_from(pm.get_source_node())

# sw = dk.get_stop_watch()
# has to enable stopwatch to use auto power measurement
# sw.start(True)
# pm.start_auto_power_measurement(100) # 100Hz

# pm.stop_reading_thread()
adc = power_module.get_digital_adc()
adc.write([samna.unifirm.adc.ina229.RegisterRead(0x7)]) #digital current
time.sleep(2)
print(adc.read())

adc.write([samna.unifirm.adc.ina229.RegisterRead(0x5)]) #digital voltage
time.sleep(2)
print(adc.read())

adc.write([samna.unifirm.adc.ina229.RegisterRead(0x4)]) #digital v_shant
time.sleep(2)
print(adc.read())

adc.write([samna.unifirm.adc.ina229.RegisterRead(0x8)]) #digital raw power
time.sleep(2)
print(adc.read())

# print(events)

# print("waiting for events")
# events = buf.get_n_events(1)
# # pm.stop_auto_power_measurement()

# counts = [0,0,0]
# sums = [0,0,0]
# avgs = [0,0,0]

# for e in events:
#     sums[e.channel] += e.value
#     counts[e.channel] += 1

# idx = 0
# for sum, count in zip(sums, counts):
#     avgs[idx] = sum/count * 1000
#     idx += 1

# print(avgs, " in mW")

model = dk.get_model()
source = samna.graph.source_to(model.get_sink_node())
sink   = samna.graph.sink_from(model.get_source_node())

source.write([samna.xyloAudio3.event.WriteRegisterValue(address=0x168, data=0x0100_0000)])


def read_register(addr):
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = addr)])
    events = sink.get_n_events(1, 3000)
    assert(len(events) == 1)
    return events[0].data

def read_all_register():
    for address in range(0x0171+1):
        data = read_register(address)
        print("", hex(address), hex(data))

# read_all_register()

0.39.5.5+gd7e31585
[]
[]
[]
[]


In [2]:
0.000546 * 1000 # mA
2.5 #V


2.5

In [1]:
import samna

# Open the device and connect to the power monitor.
board = samna.device.open_device("XyloAudio3TestBoard")
power_monitor = board.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = board.get_stop_watch()

model = board.get_model()
source = samna.graph.source_to(model.get_sink_node())
sink   = samna.graph.sink_from(model.get_source_node())

source.write([samna.xyloAudio3.event.WriteRegisterValue(address=0x168, data=0x0100_0000)])

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Auto power measurement streams PowerMeasurement events continuously.
# To capture 3 seconds of data from 3 channels at 100Hz, we collect 900 events.
events = sink_pm.get_n_events(900, timeout=4000)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]
print(avgs)

def read_register(addr):
    source.write([samna.xyloAudio3.event.ReadRegisterValue(address = addr)])
    events = sink.get_n_events(1, 3000)
    assert(len(events) == 1)
    return events[0].data

def read_all_register():
    for address in range(0x0171+1):
        data = read_register(address)
        print("", hex(address), hex(data))

read_all_register()



[0.26563974970486015, 0.012951091313813853, 1.3816043195159484]
 0x0 0x10002
 0x1 0x1000000
 0x2 0x0
 0x3 0x0
 0x4 0x0
 0x5 0x0
 0x6 0x3
 0x7 0x0
 0x8 0x0
 0x9 0x0
 0xa 0x0
 0xb 0x0
 0xc 0x0
 0xd 0x1f60
 0xe 0x0
 0xf 0x0
 0x10 0x0
 0x11 0x10
 0x12 0x0
 0x13 0x0
 0x14 0x0
 0x15 0x3712457
 0x16 0x33231103
 0x17 0x11121000
 0x18 0x200
 0x19 0xc00
 0x1a 0x0
 0x1b 0x100
 0x1c 0x3
 0x1d 0x0
 0x1e 0x0
 0x1f 0x0
 0x20 0x0
 0x21 0x0
 0x22 0x0
 0x23 0x0
 0x24 0x0
 0x25 0x0
 0x26 0x0
 0x27 0x0
 0x28 0x0
 0x29 0x0
 0x2a 0x0
 0x2b 0x0
 0x2c 0x0
 0x2d 0x0
 0x2e 0x0
 0x2f 0x0
 0x30 0x0
 0x31 0x0
 0x32 0x0
 0x33 0x0
 0x34 0x0
 0x35 0x0
 0x36 0x0
 0x37 0x0
 0x38 0x0
 0x39 0x0
 0x3a 0x0
 0x3b 0x0
 0x3c 0x0
 0x3d 0x0
 0x3e 0x0
 0x3f 0x0
 0x40 0x0
 0x41 0x0
 0x42 0x0
 0x43 0x0
 0x44 0x0
 0x45 0x0
 0x46 0x0
 0x47 0x0
 0x48 0x0
 0x49 0x0
 0x4a 0x0
 0x4b 0x0
 0x4c 0x0
 0x4d 0x0
 0x4e 0x0
 0x4f 0x0
 0x50 0x0
 0x51 0x0
 0x52 0x0
 0x53 0x0
 0x54 0x0
 0x55 0x0
 0x56 0x0
 0x57 0x0
 0x58 0x0
 0x59 0x0
 0x5a 0x0
 0

In [1]:
import warnings
warnings.filterwarnings("ignore")

import samna
print(f'Samna version {samna.__version__}')

import numpy as np
# import matplotlib.pyplot as plt
# from rockpool.devices.xylo.syns61201 import config_from_specification as a2_config_from, mapper as a2_mapper, XyloSamna as a2_XyloSamna, XyloMonitor as a2_XyloMonitor
from rockpool.devices.xylo.syns65302 import config_from_specification as a3_config_from, mapper as a3_mapper, XyloSamna as a3_XyloSamna, XyloMonitor as a3_XyloMonitor

# from rockpool.transform.quantize_methods import channel_quantize
# from rockpool.nn.modules import LIF, Linear
# from rockpool.nn.combinators import Sequential


from rockpool.devices.xylo import find_xylo_hdks
xylo_hdks, xylo_support_modules, xylo_versions = find_xylo_hdks()
print(xylo_hdks, xylo_support_modules, xylo_versions)

xa3_node = ""

for version, xylo in zip(xylo_versions, xylo_hdks):
    if version == "syns65302":
        xa3_node = xylo

print(xa3_node) if xa3_node else print("no xylo audio v3 was found")

input_data = np.loadtxt('input_data.txt')
input_data = np.array(input_data, dtype=bool)

if xa3_node:
    a3_config = samna.xyloAudio3.configuration.XyloConfiguration()
    a3_config.operation_mode = samna.xyloAudio3.OperationMode.AcceleratedTime
    a3_config.input_source = samna.xyloAudio3.InputSource.Saer
    a3_config.debug.event_input_enable = True

    dt = 0.01
    Xmod = a3_XyloSamna(xa3_node, a3_config)    
    out, _, rec = Xmod.evolve(input_data, record = True, record_power = True)

    print(len(rec['io_power']))
    print(len(rec['analog_power']))
    print(len(rec['digital_power']))

    print(rec['io_power'])
    print(rec['analog_power'])
    print(rec['digital_power'])
    
    io_power = np.mean(rec['io_power'])
    analog = np.mean(rec['analog_power'])
    digital = np.mean(rec['digital_power'])


    print(f'XyloAudio 3\nAll IO:\t\t{io_power * 1e4:.1f} mW\nAFE core:\t{analog * 1e4:.1f} mW\nSNN core logic:\t{digital*1e4:.1f} mW')


Samna version 0.39.5.5+gd7e31585
from samna.xyloA3.configuration import InputInterfaceConfig
The connected Xylo HDK contains a XyloAudio 3. Importing `rockpool.devices.xylo.syns65302`
[<samna.xyloAudio3.XyloAudio3TestBoard object at 0x7ef3a3fa4770>] [<module 'rockpool.devices.xylo.syns65302' from '/home/vleite/SynSense/rockpool/rockpool/devices/xylo/syns65302/__init__.py'>] ['syns65302']
0
0
0
[]
[]
[]
XyloAudio 3
All IO:		nan mW
AFE core:	nan mW
SNN core logic:	nan mW


In [9]:
digital_power = [0.00467556, 0.00467529, 0.00463218, 0.00470295, 0.00462871]
io_power = [0.00198491, 0.0018855 , 0.00189263, 0.00194226, 0.00190522]
analog_power = [3.70605485e-05, 1.82373060e-05, 5.60791050e-05, 8.59863358e-05]

io = np.mean(io_power)
analog = np.mean(analog_power)
digital = np.mean(digital_power)
print(digital)
print(f'XyloAudio 3\nAll IO:\t\t{io * 1000:.1f} mW\nAFE core:\t{analog * 1000:.1f} mW\nSNN core logic:\t{digital*1000:.1f} mW')



0.004662938
XyloAudio 3
All IO:		1.9 mW
AFE core:	0.0 mW
SNN core logic:	4.7 mW
